# **Datenbank**

Mithilfe dieses Notebooks wird das PreProcessing der Messdaten durchgeführt. Dabei werden z.B. die Signale für die Anwesenheit in den Wohneinheiten, die Heizungsaktivierung erstellt sowie Ausreißer und Messfehler behoben. Im Anschluss werden die überarbeiteten Datensätze in eine schnell-lesbare "Pickle"-Datenbank überführt. 

>Hinweis: Vorab müssen die Messdaten über das Skript eb-scripts von der Dropbox geladen werden.

## Importiere notwenige Packages

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 5)

%load_ext autoreload
%autoreload 2

import RomansThesis as RT

Lade WetterDB...
Lade Raumklima...
Lade gespeicherte Datenbank...                                                                


## Sollen die Datenbanken aktualisiert oder nur geladen werden?

`_update = True` 

> erstellt eine neue Datenbank aus den Messdaten

`_update = False` 
> 
> lädt die bestehende Pickle-Datei

In [2]:
_update = True

## **Raumklima**

Mithilfe dieser Funktion werden die Messdaten der Raumklimamessungen via Tinkerforge-Sensoren ausgelesen.

In [4]:
IND = RT.getIND(update = _update)
IND

Aktualisiere Raumklima...                                                                    


LB                      \
                                            DA                       
                                            WE                       
                          Pyrano_Diffuse W/m^2 Pyrano_Direct W/m^2   
Datetime                                                             
2021-02-08 11:00:00+01:00                  NaN                 NaN   
2021-02-08 11:01:00+01:00                  NaN                 NaN   
..                                         ...                 ...   
2022-10-10 00:08:00+02:00                 0.24                0.12   
2022-10-10 00:09:00+02:00                 0.24                0.12   

                                                                          \
                                                                           
                                                                           
                          Pyrano_Global W/m^2 trh1_RH (%) trh1_Tair (°C)   
Datetime                                                                   
2021-02-08 11:00:00+01:00                 NaN         NaN            NaN   
2021-02-08 11:01:00+01:00                 NaN         NaN            NaN   
..                                        ...         ...            ...   
2022-10-10 00:08:00+02:00                 NaN       65.66          13.11   
2022-10-10 00:09:00+02:00                 NaN       65.66          13.11   

                                                                     \
                                                                  N   
                                                                  B   
                          trh2_RH (%) trh2_Tair (°C) n_pt_Tair (°C)   
Datetime                                                              
2021-02-08 11:00:00+01:00         NaN            NaN            NaN   
2021-02-08 11:01:00+01:00         NaN            NaN            NaN   
..                                ...            ...            ...   
2022-10-10 00:08:00+02:00         NaN            NaN          21.40   
2022-10-10 00:09:00+02:00         NaN            NaN          21.42   

                                                      ...         MW  \
                                                      ...          S   
                                                      ...         WE   
                          n_pt_Thk (°C) n_trh_RH (%)  ... E-Pow3 (W)   
Datetime                                              ...              
2021-02-08 11:00:00+01:00           NaN          NaN  ...        NaN   
2021-02-08 11:01:00+01:00           NaN          NaN  ...        NaN   
..                                  ...          ...  ...        ...   
2022-10-10 00:08:00+02:00         20.72        91.69  ...        NaN   
2022-10-10 00:09:00+02:00         20.76        91.69  ...        NaN   

                                                                         \
                                                                          
                                    WZ                                    
                          n_trh_RH (%) n_trh_Tair (°C) Fenster Ost [XL]   
Datetime                                                                  
2021-02-08 11:00:00+01:00          NaN             NaN              NaN   
2021-02-08 11:01:00+01:00          NaN             NaN              NaN   
..                                 ...             ...              ...   
2022-10-10 00:08:00+02:00          NaN             NaN              NaN   
2022-10-10 00:09:00+02:00          NaN             NaN              NaN   

                                                                          \
                                                                           
                                                                           
                          pt_Tsk (°C) s_md (0=No, 1=Yes) Fenster Süd [M]   
Datetime                                                                   
2021-02-08 11:00:00

## **Wetterdaten**

Mithilfe dieser Funktion werden die Daten der Wetterstation ausgelesen. Dabei werden folgende Operationen durchgeführt:

1. Lade TinkerForge (roh-)Datenbank
   - Setze Zeitzone (Berlin/Amsterdam)
   - Wähle Messwerte:
      `  ['ID', 'T_amb', 'Rh_amb','windspeed','gustspeed','rain','winddir','btry']`
2. Lade Daten der Pyranometer
   - Korrigiere falsche Direktstrahlung und daraus resultierende Diffusestrahlung:
   
      > Ersetze negative Werte der Direktstralung mit 0
      > 
      > Berechne die Diffusstrahlung neu:
      > 
      >  $r_{diff} = r_{global} - r_{direct}$
3. exportiere Datenbank als .pickle Datei

In [5]:
AMB = RT.getAMB(update = _update)
AMB

Aktualisiere WetterDB...                                                              


,T_amb,Rh_amb,windspeed,gustspeed,rain,winddir,global,direct,diffuse
Datetime,,,,,,,,,
2021-03-08 11:18:00+01:00,5.0,56.0,0.6,1.0,225.3,SW,0.24,0.00,0.24
2021-03-08 11:19:00+01:00,5.1,56.0,1.3,1.7,225.3,S,0.24,0.00,0.24
...,...,...,...,...,...,...,...,...,...
2022-10-10 00:08:00+02:00,8.4,99.0,0.0,0.3,262.5,NE,0.36,0.12,0.24
2022-10-10 00:09:00+02:00,8.4,99.0,0.3,0.6,262.5,NE,0.36,0.12,0.24


## **Gesamtdatenbank**

Mithilfe dieser Funktion wird das PreProcessing der gesammelten Messdaten durchgeführt und die erstellen Datensätze in einer neuen - schnell lesbaren - Datenbank abgelegt. Diese Datenbank ist die Grundlage für alle weiteren Analysen.

Dabei werden folgende Operationen durchgeführt:

1. Lüftungsverhalten
   - Extrahiere die Nutzer:innen-Interaktion mit den Fenster. Sprich wann wird ein Fenster Geöffnet (1) und wann geschloßen (-1).
   - Clustere die Signale der Fenster für Räume mit mehr als einem Fenster in ein einzelnes Signal
2. Bestimme die Anwesenheit pro Wohnheit
   - Summe der Kriterien:
      - Bei Auslösung der **Bewegungsmelder** wird 15min vorher und 15min danach Anwesenheit angenommen.
      - Wenn die **CO2-Konzantration*** steigt, wird Anwesenheit angenommen (`CO2_A`).
      - Die **CO2- Konzentration*** ist bei geschlossenen Fenstern mehr als 10% über der ref. Konzentration im unbewohnten Appartment (`CO2_B`).
      - Wenn bei geöffneten Fenstern der 15min gleitende Mittelwert der **CO2-Konzentration*** größer als das Tagesmittel ist, wird Anwesenheit angenommen (relevant für Sommerfall bei geöffneten Fenstern) (`CO2_B`)
      - Steigt die **absolute Luftfeuchte** in einem kurzen Zeitraum stark an, wird Anwesenheit angenommen
      - Werden **Fenster** geöffnet oder geschlossen, wird Anwesenheit angenommen.
      - Wird **Abends** (nach 22:00 Uhr) Anwesenheit aufgrund eines der o.g. Kriterien festgestellt, wird angenommen, dass bis zum nächten Morgen jemand im Appartment anwesend ist.
3. Extrahiere Messwerte aus den TinkerForge Datenbanken
   - Wärmemengenzähler
   - Temperaturen (Schwarzkugel, Luft)
   - rel. Luftfeuchten 
   - CO2-Konzentration
   - Außenklima (Temp, Rh, windspeed, global)
4. Berechne die operative Temperatur aus der Schwarzkugeltemperatur
5. Berechne die absoulte Luftfeuchte aus der Lufttemperatur und rel. Feuchte
6. Bestimme die Heizkörperklassifikation
7. Entferne Unbewohnte Zeiträume

In [6]:
DB = RT.getDB(update = _update, correction='lin')
DB

Aktualisiere Datenbank...
#1/9: Leichtbeton Nord
|----> Fertig! Dauer: 59 sec.                                                                                                              
#2/9: Leichtbeton Ost
|----> Fertig! Dauer: 19 sec.                                                                                                              
#3/9: Leichtbeton Süd
|----> Fertig! Dauer: 45 sec.                                                                                                              
#4/9: Massivholz Nord
|----> Fertig! Dauer: 34 sec.                                                                                                              
#5/9: Massivholz Ost
|----> Fertig! Dauer: 17 sec.                                                                                                              
#6/9: Massivholz Süd
|----> Fertig! Dauer: 40 sec.                                                                                                              
#7/

bui                          LB                                           \
app                           N                                            
room                          B                                            
value                     HK_IO  Rh Rh_amb T_amb T_amb_g24 Tair Thk Tset   
Datetime                                                                   
2021-02-08 11:00:00+01:00   NaN NaN    NaN   NaN       NaN  NaN NaN  NaN   
2021-02-08 11:01:00+01:00   NaN NaN    NaN   NaN       NaN  NaN NaN  NaN   
...                         ...  ..    ...   ...       ...  ...  ..  ...   
2022-10-10 01:59:00+02:00   NaN NaN    NaN   NaN       NaN  NaN NaN  NaN   
2022-10-10 02:00:00+02:00   NaN NaN    NaN   NaN       NaN  NaN NaN  NaN   

bui                                      ...  MW                              \
app                                      ...   S                               
room                                  K  ...  WZ                               
value                     g_abs Fenster  ...  Rh Rh_amb T_amb T_amb_g24 Tair   
Datetime                                 ...                                   
2021-02-08 11:00:00+01:00   NaN     NaN  ... NaN    NaN   NaN       NaN  NaN   
2021-02-08 11:01:00+01:00   NaN     NaN  ... NaN    NaN   NaN       NaN  NaN   
...                         ...     ...  ...  ..    ...   ...       ...  ...   
2022-10-10 01:59:00+02:00   NaN     NaN  ... NaN    NaN   NaN       NaN  NaN   
2022-10-10 02:00:00+02:00   NaN     NaN  ... NaN    NaN   NaN       NaN  NaN   

bui                                               
app                                               
room                                              
value                     Thk Top Tset Tsk g_abs  
Datetime                                          
2021-02-08 11:00:00+01:00 NaN NaN  NaN NaN   NaN  
2021-02-08 11:01:00+01:00 NaN NaN  NaN NaN   NaN  
...                        ..  ..  ...  ..   ...  
2022-10-10 01:59:00+02:00 NaN NaN  NaN NaN   NaN  
2022-10-10 02:00:00+02:00 NaN NaN  NaN NaN   NaN  

[876361 rows x 408 columns]

## **Lüftungsverhalten**

Diese Funktion erkennt Lüftungsvorgänge in den Messdaten und sammelt Kenndaten wie:
- Zeitraum
- Temperaturen beim öffnen und Schließen, Höchst- und Tiefsttemperaturen, Mittelwerte etc.

In [7]:
LüftungsDB = RT.createLüftungsDB(update = _update)
LüftungsDB

Aktualisiere LüftungsDB...


value                                Datetime                            \
                                        first                      last   
bui app room id                                                           
LB  N   SZ   1.0    2021-03-08 11:18:00+01:00 2021-03-08 14:41:00+01:00   
             2.0    2021-03-08 14:42:00+01:00 2021-03-09 13:22:00+01:00   
...                                       ...                       ...   
MW  S   WZ   1334.0 2022-10-08 22:46:00+02:00 2022-10-08 23:10:00+02:00   
             1335.0 2022-10-08 23:11:00+02:00 2022-10-10 00:00:00+02:00   

value                               Fenster  Tair                           \
                          timedelta    mean first  last   min   max median   
bui app room id                                                              
LB  N   SZ   1.0    0 days 03:23:00     1.0  20.3  20.4  19.4  21.2   20.7   
             2.0    0 days 22:40:00     0.0  20.4  21.3  20.4  21.6   20.9   
...                             ...     ...   ...   ...   ...   ...    ...   
MW  S   WZ   1334.0 0 days 00:24:00     1.0  21.0  20.6  20.6  21.0   20.7   
             1335.0 1 days 00:49:00     0.0  20.7  20.9  19.9  21.4   21.0   

value                      ... T_amb_g24                    Rh_amb        \
                     mean  ...       min   max median  mean  first  last   
bui app room id            ...                                             
LB  N   SZ   1.0     20.7  ...       5.0   7.6    7.0   6.8   56.0  51.0   
             2.0     21.0  ...       0.6   7.5    3.0   3.5   51.0  41.0   
...                   ...  ...       ...   ...    ...   ...    ...   ...   
MW  S   WZ   1334.0  20.8  ...      12.7  12.7   12.7  12.7   99.0  99.0   
             1335.0  21.0  ...      10.4  12.9   12.2  12.0   99.0  99.0   

value                                         
                      min   max median  mean  
bui app room id                               
LB  N   SZ   1.0     37.0  56.0   41.0  44.9  
             2.0     37.0  89.0   73.0  71.1  
...                   ...   ...    ...   ...  
MW  S   WZ   1334.0  99.0  99.0   99.0  99.0  
             1335.0  73.0  99.0   98.0  92.4  

[20809 rows x 59 columns]

## **Thermischer Komfort**

Mithilfe dieser Funktion wird der Predicted Mean Vote (PMV) sowie die projected percentage of dissatified (PPD) berechnet

In [8]:
ThermalComfort, pmv_params = RT.calculatePMVPPD(update = _update)
ThermalComfort

Berechne PMV und PPD für Schlafzimmer und Wohnzimmer


bui                          LB                                                \
app                           N                                                 
room                         SZ                                                 
value                      Tair   Top    Rh  met  clo  vair v_rel clo_d   pmv   
Datetime                                                                        
2021-02-09 11:00:00+01:00   NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN   NaN   
2021-02-09 12:00:00+01:00   NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN   NaN   
...                         ...   ...   ...  ...  ...   ...   ...   ...   ...   
2022-10-09 23:00:00+02:00  22.1  22.1  81.1  1.0  1.0  0.15  0.15   1.0 -0.14   
2022-10-10 00:00:00+02:00  22.1  22.1  81.1  1.0  1.0  0.15  0.15   1.0 -0.14   

bui                             ...     MW                                     \
app                             ...      S                                      
room                            ...     WZ                                      
value                      ppd  ...   Tair   Top     Rh  met  clo  vair v_rel   
Datetime                        ...                                             
2021-02-09 11:00:00+01:00  NaN  ...    NaN   NaN    NaN  NaN  NaN   NaN   NaN   
2021-02-09 12:00:00+01:00  NaN  ...    NaN   NaN    NaN  NaN  NaN   NaN   NaN   
...                        ...  ...    ...   ...    ...  ...  ...   ...   ...   
2022-10-09 23:00:00+02:00  5.4  ...  21.00  21.0  74.30  1.2  1.0  0.15  0.21   
2022-10-10 00:00:00+02:00  5.4  ...  20.95  21.0  74.55  1.2  1.0  0.15  0.21   

bui                                         
app                                         
room                                        
value                     clo_d   pmv  ppd  
Datetime                                    
2021-02-09 11:00:00+01:00   NaN   NaN  NaN  
2021-02-09 12:00:00+01:00   NaN   NaN  NaN  
...                         ...   ...  ...  
2022-10-09 23:00:00+02:00   1.0 -0.17  5.6  
2022-10-10 00:00:00+02:00   1.0 -0.18  5.7  

[14581 rows x 150 columns]